In [1]:
import numpy as np
import matplotlib.pyplot as plt

DIFFUSIVITY = 0.6 #  In [fm]
r_0 = 1.2 #  In [fm]
NUM_CORE_NUCLEONS = 10
TOT_ANG_MOMENTUM = 0.5
ORB_ANG_MOMENTUM = 0
V_LS = - 21.0 # In MeV, we keep all - signs in the defenition of the parameters
NUM_GAUSSIANS = 6
# EXPANSION_COEFFICIENT_INITAL_GUESSES = [0.1, 0.1, 0.1, 0.1]
STARTING_POTENTIAL_PARAMETER = 0.05

In [14]:
def gaussian_function(r, c, β):
    return c * np.exp(- β * (r)**2)

def woods_saxon_function(r, r_0=r_0, A_c=NUM_CORE_NUCLEONS, a=DIFFUSIVITY):
    R_0 = r_0 * A_c**(1/3)
    return 1 / (1 + np.exp((r - R_0) / a))

def woods_saxon_function_derivative(r, r_0=r_0, A_c=NUM_CORE_NUCLEONS, a=DIFFUSIVITY):
    R_0 = r_0 * A_c**(1/3)
    numerator = np.exp((r - R_0) / a)
    denominator = a * (np.exp((r - R_0) / a) + 1)**2
    return -1 * numerator * denominator

def spin_orbit_coefficent(j, l):
    return (j * (j + 1) - l * (l + 1) - 3 / 4 ) / 2

def geometric_progression(potential_parameter, i, geometric_progression_number=2):
    return potential_parameter * geometric_progression_number**(i)

def full_gaussian_potential(r, expansion_coefficients, sum_limit=NUM_GAUSSIANS, potential_parameter=STARTING_POTENTIAL_PARAMETER):
    """
    Gives the value of our gaussian potential for a given values of r. 

    Parameters
    ----------
    r : array like
        DESCRIPTION.
    expansion_coefficients : 1D array
        should be the same length as sum_limit
    sum_limit : intger, optional
        Specifies the number of gaussians we take the expansion over

    Returns
    -------
    potential : array like
        the potential at a (set of) points r.

    """
    potential = 0
    for i in range(sum_limit):
        potential_parameter = geometric_progression(potential_parameter, i)
        potential += gaussian_function(r, expansion_coefficients[i], potential_parameter)
    return potential


def full_ws_potential(r, tot_ang_momentum, orb_ang_momentum, V_ls=V_LS):
    potential_depth = -11.39 * (-1)**orb_ang_momentum - 51.13
    return potential_depth * woods_saxon_function(r) + V_ls * spin_orbit_coefficent(tot_ang_momentum, orb_ang_momentum) * (1 / r) * woods_saxon_function_derivative(r)

We need to specify our matricies to solve by a least squares method, chose numpy.linalg.lstsq. Specify the functions in each (dont have to sum over sample points, lstsq does this already) in a form A * C = B (A is a N x N matrix, B, C, is a N dimension vector).

In [23]:
r_values = np.linspace(0.000000000001, 12, 1000)
NUMBER_OF_GAUSSIANS = 5
ratio = 2
beta_1 = 0.05
betas = beta_1 * ratio ** np.arange(NUMBER_OF_GAUSSIANS)
V0 = np.abs(-11.39 * (-1)**ORB_ANG_MOMENTUM - 51.13)

A = V0 * np.exp(-(r_values[:, None]**2) * betas[None, :])
#print(len(A[0, :]))

#vector b
R0 = r_0 * (NUM_CORE_NUCLEONS ** (1/3))
b = full_ws_potential(r_values, TOT_ANG_MOMENTUM, ORB_ANG_MOMENTUM)
# print(b)

In [25]:
least_squares_matrix = np.zeros(shape=(NUM_GAUSSIANS,NUM_GAUSSIANS))
least_squares_vector = np.zeros(shape=(NUM_GAUSSIANS, 1))
r_vals = np.linspace(0.0000000001, 15, 1000000)
i_potential_parameter = STARTING_POTENTIAL_PARAMETER
j_potential_parameter = STARTING_POTENTIAL_PARAMETER

for i in range(NUM_GAUSSIANS):
    i_potential_parameter = geometric_progression(i_potential_parameter, i)
    for j in range(NUM_GAUSSIANS):
        j_potential_parameter = geometric_progression(j_potential_parameter, j)
        least_squares_matrix[i, j] = np.exp(- i_potential_parameter * (r_vals)**2) *  np.exp(- j_potential_parameter * (r_vals)**2)
        least_squares_matrix[i, j] = np.exp(- (i_potential_parameter + j_potential_parameter) * (r_vals)**2)
    least_squares_vector[i, 0] = np.exp(- i_potential_parameter * (r_vals)**2) * full_ws_potential(r_vals)

print(least_squares_matrix)
print(least_squares_vector)

ValueError: setting an array element with a sequence.